In [1]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: C:\Users\Dell\.virtualenvs\Week_4-XdYpKifU\Lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 


In [2]:
import pickle
import pandas as pd
import numpy as np
import os

In [3]:
year = 2022
month = 2

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'./output/yellow_taxi_tripdata_{year:04d}-{month:02d}.parquet'

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
y_pred.std()

5.28140357655334

In [9]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [10]:
!stat -f '%z' output_file

'stat' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
file_size = os.path.getsize(output_file)

size_in_mb = file_size / 1024 / 1000

print(f"The size of the file is approximately: {size_in_mb:.2f} MB")

The size of the file is approximately: 58.59 MB
